# Probing part 1

Vantage points will probe either the targets themselves (step 2) or other addresses in the same /24 prefix (step 2).

Vantage points are only the anchors.  
As always, targets are the anchors.  

This notebook is an implementation of the million scale method. Check the paper for more information.  
To do after create_datasets.ipynb

In [1]:
import uuid

from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    load_targets, 
    load_vps,
    get_measurement_config,
    save_measurement_config,
    get_target_prefixes,
    ping_prefixes,
    ping_targets,
    retrieve_results,
)
from default import (
    USER_ANCHORS_FILE, 
    USER_HITLIST_FILE, 
    PREFIX_MEASUREMENT_RESULTS,
    TARGET_MEASUREMENT_RESULTS,
    MEASUREMENT_CONFIG_PATH,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

## Load targets and vps dataset

In [2]:

targets = load_targets(USER_ANCHORS_FILE, nb_target=NB_TARGETS)
vps = load_vps(USER_ANCHORS_FILE, nb_vps=NB_VPS)

# all target prefixes
target_prefixes = get_target_prefixes(targets)

# responsive IP addresses in all /24 prefixes
targets_per_prefix = load_json(USER_HITLIST_FILE)

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

2023-09-13 10:18:49::INFO:root:2498612375:: nb targets: 2
2023-09-13 10:18:49::INFO:root:2498612375:: nb_vps : 4


## Generate measurement config

This configuration is used to retrieve all measurements results from RIPE Atlas using their API.

In [3]:
# measurement configuration for retrieval
experiment_uuid = uuid.uuid4()
target_measurement_uuid = uuid.uuid4()
prefix_measurement_uuid = uuid.uuid4()

measurement_config = get_measurement_config(
    experiment_uuid=experiment_uuid,
    target_measurement_uuid=target_measurement_uuid,
    prefix_measurement_uuid=prefix_measurement_uuid,
    targets=targets, 
    target_prefixes=target_prefixes,
    vps=vps,
)

save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)

# Step 1: probing each target prefixes

## Probe target prefixes
WARNING : Time consumming section

In [4]:
ping_prefixes(
    measurement_uuid=prefix_measurement_uuid,
    measurement_config=measurement_config,
    target_prefixes=target_prefixes,
    targets_per_prefix=targets_per_prefix,
    vps=vps,
)

save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)

2023-09-13 10:18:49::INFO:root:measurement_utils:: No cached results available
2023-09-13 10:18:49::INFO:root:measurement_utils:: Starting measurements f29ff03d-ede7-44ca-8fc8-fe2f832fd57a with parameters: dry_run=False; nb_targets=5803825; nb_vps=4.


vps list [7139, 6749, 6515, 7031]
vps list [7139, 6749, 6515, 7031]
vps list [7139, 6749, 6515, 7031]
vps list [7139, 6749, 6515, 7031]
vps list [7139, 6749, 6515, 7031]
vps list [7139, 6749, 6515, 7031]


2023-09-13 10:18:52::INFO:root:ping_and_traceroute_classes:: measurement : f29ff03d-ede7-44ca-8fc8-fe2f832fd57a done


# Step 2: probing each target

## Probe targets
WARNING : Time consumming section

In [5]:
# measurement configuration for retrieval
ping_targets(
    measurement_uuid=prefix_measurement_uuid,
    measurement_config=measurement_config,
    targets=targets,
    vps=vps,
)
save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)


2023-09-13 10:18:53::INFO:root:measurement_utils:: No cached results available
2023-09-13 10:18:53::INFO:root:measurement_utils:: Starting measurements f29ff03d-ede7-44ca-8fc8-fe2f832fd57a with parameters: dry_run=False; nb_targets=2; nb_vps=4.
2023-09-13 10:18:54::INFO:root:ping_and_traceroute_classes:: measurement : f29ff03d-ede7-44ca-8fc8-fe2f832fd57a done


## Retrieve prefix results
WARNING : Time consuming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [6]:
# TODO: get latest measurement available
retrieve_results(measurement_config, PREFIX_MEASUREMENT_RESULTS)

KeyError: 'UUID'

## Retrieve traget results
WARNING : Time consumming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [ ]:
retrieve_results(measurement_config, TARGET_MEASUREMENT_RESULTS)

2023-09-12 20:05:54::INFO:root:measurement_utils:: nb measurements retrieved: 2
2023-09-12 20:05:54::INFO:root:measurement_utils:: target: 45.239.44.43 | number of measurement retrieved: 4
2023-09-12 20:05:54::INFO:root:measurement_utils:: target: 91.200.16.69 | number of measurement retrieved: 4
